In [1]:
import os
import cv2 as cv
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten

In [ ]:
###################################################### 실행 안함. 자동완성용

from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
######################################################

In [63]:

# 폴더 리스트 생성
folder_list = os.listdir('/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train')
folder_list.remove('.DS_Store')

# 데이터를 저장할 2차원 배열 생성
data = []
count = []

# 폴더 리스트를 순회하며
for folder_name in folder_list:
    folder_path = '/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train/' + folder_name
    #print(folder_path)
    # 폴더 내 이미지 리스트 생성
    image_list = os.listdir(folder_path)
    list_len = len(image_list)
    count.append(list_len)
    # 해당 폴더 내 이미지 리스트를 순회하며
    for image_name in image_list:
        image_path = folder_path +'/' + image_name
        
        # 이미지 로드 및 그레이스케일 적용
        img = cv.imread(image_path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        # 생성한 데이터 배열에 이미지 추가
        data.append(gray)
        
        
# 데이터 배열 확인
#print(np.array(data).shape)


In [2]:

folder_list = os.listdir('/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train')
folder_list.remove('.DS_Store') 

data = []
pic_count = []

for folder_name in folder_list:
    folder_path = '/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train/' + folder_name
  
    image_list = os.listdir(folder_path)
    list_len = len(image_list)
    pic_count.append(list_len)
    
    for image_name in image_list:
        image_path = folder_path +'/' + image_name
        img = cv.imread(image_path)
        resize = cv.resize(img, (300,300), interpolation=cv.INTER_CUBIC)
        gray = cv.cvtColor(resize, cv.COLOR_BGR2GRAY)
        data.append(gray)

In [3]:
answer = []
for idx, nm in enumerate(folder_list):
        answer += [nm] * pic_count[idx]
answer = np.array(answer)

In [5]:
answer = pd.get_dummies(answer)

In [6]:
answer = answer.values

In [7]:
answer

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False]])

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(np.array(data),answer,test_size=0.3, shuffle=True)

In [9]:
print(xtrain.shape)
print(xtest.shape)

(2419, 300, 300)
(1038, 300, 300)


In [10]:
xtrain = xtrain.reshape(2419,300,300,1) 
xtest = xtest.reshape(1038,300,300,1)

In [11]:
model = Sequential([
            Input(shape = (300,300,1), name = 'input_layer'),
            Conv2D(32,kernel_size=3, activation= 'relu', name = 'cov_layer1' ),
            #Dropout(0.5) 데이터 기반한 weight를 줄여 오버핏팅을 줄이는 것
            MaxPooling2D(pool_size= 2), 
            Flatten()
            ])

model.add(Dense(32, activation= 'relu'))
model.add(Dense(19, activation= 'softmax'))

model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cov_layer1 (Conv2D)         (None, 298, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 710432)            0         
                                                                 
 dense (Dense)               (None, 32)                22733856  
                                                                 
 dense_1 (Dense)             (None, 19)                627       
                                                                 
Total params: 22,734,803
Trainable params

In [12]:
model.compile(optimizer= 'adam', loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(xtrain,ytrain, validation_data=(xtest, ytest), batch_size=100, epochs = 10)

Epoch 1/10


2023-05-06 22:02:40.261464: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


25/25 [==============================] - 8s 254ms/step - loss: 3697.6536 - accuracy: 0.1616 - val_loss: 2.9466 - val_accuracy: 0.0645
Epoch 2/10
25/25 [==============================] - 7s 260ms/step - loss: 61.6595 - accuracy: 0.0542 - val_loss: 2.9282 - val_accuracy: 0.0645
Epoch 3/10
25/25 [==============================] - 6s 228ms/step - loss: 2.9200 - accuracy: 0.2906 - val_loss: 2.9081 - val_accuracy: 0.4143
Epoch 4/10
25/25 [==============================] - 6s 226ms/step - loss: 2.9008 - accuracy: 0.4031 - val_loss: 2.8883 - val_accuracy: 0.4143
Epoch 5/10
25/25 [==============================] - 6s 227ms/step - loss: 2.8819 - accuracy: 0.4031 - val_loss: 2.8682 - val_accuracy: 0.4143
Epoch 6/10
25/25 [==============================] - 6s 227ms/step - loss: 2.8628 - accuracy: 0.4031 - val_loss: 2.8482 - val_accuracy: 0.4143
Epoch 7/10
25/25 [==============================] - 6s 228ms/step - loss: 2.8438 - accuracy: 0.4031 - val_loss: 2.8285 - val_accuracy: 0.4143
Epoch 8/10
25

In [13]:
model.save('wall_model_ver1.hdf5')

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.3.1-arm64-arm-64bit
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Pandas 2.0.1
Scikit-Learn 1.2.2
GPU is available


In [2]:
# What version of Python do you have?
import sys
import platform
import torch
import pandas as pd
import sklearn as sk

has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-13.3.1-arm64-arm-64bit
PyTorch Version: 2.0.0

Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Pandas 2.0.1
Scikit-Learn 1.2.2
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [ ]:
def predict_image_sample_(model, X_test, y_test, test_id=-1):
    if test_id < 0:
        from random import randrange
        test_sample_id = randrange(10000)
    else:
        test_sample_id = test_id
        
    test_image = X_test[test_sample_id]
    
    #plt.imshow(test_image, cmap='gray')
    
    test_image = test_image.reshape(1,28,28,1)

    y_actual = y_test[test_sample_id]
    print("y_actual number=", y_actual)
    
    y_pred = model.predict(test_image)
    #print("y_pred=", y_pred)
    y_pred_num = np.argmax(y_pred, axis=1)[0]
    print("y_pred number=", y_pred_num)
    
    return y_actual, y_pred_num

In [ ]:
all, n = 0,0
(X_train, y_train), (X_test, y_test) = mnist.load_data()

for i in range(500):
    all +=1
    y_actual, y_pred_num = predict_image_sample_(model, X_test, y_test, test_id= i)
    if y_actual == y_pred_num:
        n += 1 
    else : pass
print(f'총횟수 = {all} , 맞춘횟수={n} , 정확도(Accuracy)={(n/all)*100}')